In [1]:
import codecs
import sys
from os import path
import os
import re
import pickle
import numpy as np
import pandas as pd
import itertools
import nltk
from nltk.probability import FreqDist
from pickle import dump,load
from gensim.models import word2vec

# SogouC.reduced.zip; https://pan.baidu.com/s/1pLrORJt
rootdir = './data/SogouC.reduced/Reduced'
dirs = os.listdir(rootdir)
dirs = [path.join(rootdir,f) for f in dirs if f.startswith('C')]


In [2]:
def load_txt(x):
    with open(x) as f:
        res = [t for t in f.readlines()]
        return ''.join(res)


# In[4]:

def to_df(dirs):
    text_t = {}
    for i, d in enumerate(dirs):
        files = os.listdir(d)
        files = [path.join(d, x) for x in files if x.endswith('txt') and not x.startswith('.')]
        text_t[i] = [load_txt(f) for f in files]

    # to dataframe

    flen = [len(t) for t in text_t.values()]


    # In[7]:


    labels = np.repeat(list(text_t.keys()),flen)


    # In[8]:


    # flatter nested list

    merged = list(itertools.chain.from_iterable(text_t.values()))


    # In[9]:


    df = pd.DataFrame({'label': labels, 'txt': merged})
    df.head()
    df['seg_word'] = df.txt.map(cutchar)

    return df

# In[10]:


# cut character
def cutchar(x):
    words = list(x)
    return ' '.join(words)

df = to_df(dirs)

In [3]:

# 探索 转成nltk需要格式,建立list
txt = df['seg_word'].values
txtnltk = []
for sent in txt:
    temp = [w for w in sent.split()]
    txtnltk.extend(temp)

# nltk
corpus = nltk.Text(txtnltk)

# 词频
fdist = FreqDist(corpus)
# w = fdist.keys()
# v = fdist.values()
w, v = [], []
for k1, v1 in fdist.items():
    w.append(k1)
    v.append(v1)
freqdf = pd.DataFrame({'word':w,'freq':v})
freqdf.sort_values('freq',ascending =False, inplace=True)
freqdf['idx'] = np.arange(len(v))
freqdf.head()

,freq,word,idx
6393,4268470, ,0
372,886090,，,1
4133,698338,的,2
3586,411205,。,3
2855,224226,一,4


In [4]:

word2idx = {c:i for c, i in zip(freqdf.word, freqdf.idx)}
idx2word = dict((i, c) for c, i in zip(freqdf.word, freqdf.idx))

print('保存word2idx到文件')
dump(word2idx, open('word2idx.pickle', 'wb'))
print('保存idx2word到文件')
dump(idx2word, open('idx2word.pickle', 'wb'))

# In[23]:


# 保持字符串，为生成词向量准备
all_news_wv = []
for news in txt:
    all_news_wv.append([x for x in news.split() ])


# In[24]:


# word2vec
def trainW2V(corpus, epochs=50, num_features = 100,
             min_word_count = 1, num_workers = 4,
             context = 10, sample = 1e-5):
    global w2v
    w2v = word2vec.Word2Vec(workers = num_workers,
                          sample = sample,
                          size = num_features,
                          min_count=min_word_count,
                          window = context)
    np.random.shuffle(corpus)
    w2v.build_vocab(corpus)
    for epoch in range(epochs):
        print(epoch, end=' ')
        np.random.shuffle(corpus)
        w2v.train(corpus, epochs=w2v.iter,total_examples=w2v.corpus_count)
        w2v.alpha *= 0.9
        w2v.min_alpha = w2v.alpha
    print("Done.")


# In[25]:


# word2vec
trainW2V(all_news_wv)

保存word2idx到文件
保存idx2word到文件
0 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 Done.


In [5]:
# 保存词向量lookup矩阵，按idx位置存放。目的是保存词频，也可以直接使用w2v.index2word
init_weight_wv = []
for i in range(freqdf.shape[0]):
    init_weight_wv.append(w2v[idx2word[i]])


# In[15]:

print('保存字向量到文件')
dump(init_weight_wv, open('init_weight.pickle', 'wb'))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


保存字向量到文件


In [6]:

# 读取数据，将格式进行转换为带四种标签 S B M E
input_file = './data/icwb2-data/training/msr_training.utf8'
output_file = './data/icwb2-data/training/msr_training.tagging.utf8'


# 用于字符标记的4个标签：B（开始），E（结束），M（中），S（单）

def character_tagging(input_file, output_file):
    input_data = codecs.open(input_file, 'r', 'utf-8')
    output_data = codecs.open(output_file, 'w', 'utf-8')
    for line in input_data.readlines():
        word_list = line.strip().split()
        for word in word_list:
            if len(word) == 1:
                output_data.write(word + "/S ")
            else:
                output_data.write(word[0] + "/B ")
                for w in word[1:len(word)-1]:
                    output_data.write(w + "/M ")
                output_data.write(word[len(word)-1] + "/E ")
        output_data.write("\n")
    input_data.close()
    output_data.close()

character_tagging(input_file, output_file)



# 定义'U'为未登陆新字, 空格为两头padding用途，并增加两个相应的向量表示
char_num = len(init_weight_wv)
idx2word[char_num] = u'U'
word2idx[u'U'] = char_num
idx2word[char_num+1] = u' '
word2idx[u' '] = char_num+1

init_weight_wv.append(np.random.randn(100,))
init_weight_wv.append(np.zeros(100,))


# In[21]:


# 分离word 和 label
with open(output_file) as f:
    lines = f.readlines()
    train_line = [[w[0] for w in line.split()] for line in lines]
    train_label = [w[2] for line in lines for w in line.split()]


# In[17]:


# 文档转数字list
def sent2num(sentence, word2idx = word2idx, context = 7):
    predict_word_num = []
    for w in sentence:
        # 文本中的字如果在词典中则转为数字，如果不在则设置为'U
        if w in word2idx:
            predict_word_num.append(word2idx[w])
        else:
            predict_word_num.append(word2idx[u'U'])
    # 首尾padding
    num = len(predict_word_num)
    pad = int((context-1)*0.5)
    for i in range(pad):
        predict_word_num.insert(0,word2idx[u' '] )
        predict_word_num.append(word2idx[u' '] )
    train_x = []
    for i in range(num):
        train_x.append(predict_word_num[i:i+context])
    return train_x


# In[53]:


# 输入字符list，输出数字list
sent2num(train_line[0])


[[6892, 6892, 6892, 19, 11, 52, 222],
 [6892, 6892, 19, 11, 52, 222, 81],
 [6892, 19, 11, 52, 222, 81, 31],
 [19, 11, 52, 222, 81, 31, 276],
 [11, 52, 222, 81, 31, 276, 5],
 [52, 222, 81, 31, 276, 5, 4],
 [222, 81, 31, 276, 5, 4, 84],
 [81, 31, 276, 5, 4, 84, 189],
 [31, 276, 5, 4, 84, 189, 220],
 [276, 5, 4, 84, 189, 220, 411],
 [5, 4, 84, 189, 220, 411, 1],
 [4, 84, 189, 220, 411, 1, 78],
 [84, 189, 220, 411, 1, 78, 703],
 [189, 220, 411, 1, 78, 703, 106],
 [220, 411, 1, 78, 703, 106, 664],
 [411, 1, 78, 703, 106, 664, 2],
 [1, 78, 703, 106, 664, 2, 137],
 [78, 703, 106, 664, 2, 137, 430],
 [703, 106, 664, 2, 137, 430, 224],
 [106, 664, 2, 137, 430, 224, 5],
 [664, 2, 137, 430, 224, 5, 10],
 [2, 137, 430, 224, 5, 10, 57],
 [137, 430, 224, 5, 10, 57, 62],
 [430, 224, 5, 10, 57, 62, 87],
 [224, 5, 10, 57, 62, 87, 2],
 [5, 10, 57, 62, 87, 2, 189],
 [10, 57, 62, 87, 2, 189, 220],
 [57, 62, 87, 2, 189, 220, 411],
 [62, 87, 2, 189, 220, 411, 1],
 [87, 2, 189, 220, 411, 1, 319],
 [2, 189, 2

In [7]:

# 将所有训练文本转成数字list
train_word_num = []
for line in train_line:
    train_word_num.extend(sent2num(line))


# In[62]:


print(len(train_word_num))
print(len(train_label))

4050469
4050469


In [8]:
import numpy as np
from pickle import dump,load
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.core import Reshape, Flatten ,Dropout
from sklearn.model_selection import train_test_split
from keras.callbacks import Callback



Using TensorFlow backend.


In [9]:

nb_classes = len(np.unique(train_label))

# 建立两个字典
label_dict = dict(zip(np.unique(train_label), range(4)))
num_dict = {n:l  for l,n  in label_dict.items()}
print(label_dict)
print(num_dict)
# 将目标变量转为数字
train_label = [label_dict[y] for y in train_label]


# In[70]:


# 切分数据集
train_X, test_X, train_y, test_y = train_test_split(train_word_num, train_label , train_size=0.9, random_state=1)


# In[71]:


Y_train = np_utils.to_categorical(train_y, nb_classes)
Y_test = np_utils.to_categorical(test_y, nb_classes)


# In[72]:


print(len(train_X), 'train sequences')
print(len(test_X), 'test sequences')

{'M': 2, 'B': 0, 'E': 1, 'S': 3}
{0: 'B', 1: 'E', 2: 'M', 3: 'S'}


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


3645422 train sequences
405047 test sequences


In [10]:


# 初始字向量格式准备
init_weight = [np.array(init_weight_wv)]


# In[79]:


batch_size = 256


# In[75]:


maxfeatures = init_weight[0].shape[0] # 词典大小


# In[87]:


# 一个普通的单隐层神经网络，输入层700，隐藏层100，输出层4
# 迭代时同时更新神经网络权重，以及词向量
print('Build model...')
model = Sequential()
# 词向量初始化，输入维度：词典大小，输出维度：词向量100
model.add(Embedding(maxfeatures, 100,weights=init_weight,input_length=7)) # 使用初使词向量可以增加准确率
# Embedding: 将正整数（索引值）转换为固定尺寸的稠密向量。 例如： [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]; 该层只能用作模型中的第一层。

model.add(Flatten())
# Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。Flatten不影响batch的大小。
# eg: (None, 64, 32, 32) --> (None, 65536)

model.add(Dense(100, input_dim=700, activation='relu'))
model.add(Dropout(0.5))  # 为输入数据施加Dropout。Dropout将在训练过程中每次更新参数时按一定概率（rate）随机断开输入神经元，Dropout层用于防止过拟合。

# model.add(Activation('relu'))
model.add(Dense(nb_classes, input_dim=100, activation='softmax'))
# model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 100)            689300    
_________________________________________________________________
flatten_1 (Flatten)          (None, 700)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               70100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total params: 759,804
Trainable params: 759,804
Non-trainable params: 0
_________________________________________________________________


In [11]:

class EarlyStopping(Callback):
    def __init__(self, patience=0, verbose=0):
        super(Callback, self).__init__()

        self.patience = patience
        self.verbose = verbose
        self.best_val_loss = np.Inf
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        # if not self.params['do_validation']:
        #     warnings.warn("Early stopping requires validation data!", RuntimeWarning)

        cur_val_loss = logs.get('val_loss')
        if cur_val_loss < self.best_val_loss:
            self.best_val_loss = cur_val_loss
            self.wait = 0
        else:
            if self.wait >= self.patience:
                if self.verbose > 0:
                    print("Epoch %05d: early stopping" % (epoch))
                self.model.stop_training = True
            self.wait += 1


# In[144]:


# train_X, test_X, Y_train, Y_test
print("Train...")
earlystop = EarlyStopping(patience=0, verbose=1)
result = model.fit(np.array(train_X), Y_train, batch_size=batch_size, nb_epoch=1,
          validation_split=0.1,callbacks=[earlystop])


# In[145]:


score = earlystop.model.evaluate(np.array(test_X), Y_test, batch_size=batch_size)
print('Test score:', score)


# In[146]:


# test数据集，准确率0.94
classes = earlystop.model.predict_classes(np.array(test_X), batch_size=batch_size)
# acc = np_utils.accuracy(classes, test_y) # 要用没有转换前的y
acc = sum([1 if k==v else 0 for k, v in zip(classes, test_y)])/len(test_y)
print('Test accuracy:', acc)


Train...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 3280879 samples, validate on 364543 samples
Epoch 1/1
405047/405047 [==============================] - 2s 4us/step
Test score: 0.3393762092336496
Test accuracy: 0.8757773789214585


In [13]:

import numpy as np
from pickle import dump,load


# - 步骤4：用test文本进行预测，评估效果

temp_txt = u'国家食药监总局发布通知称，酮康唑口服制剂因存在严重肝毒性不良反应，即日起停止生产销售使用。'
temp_txt = list(temp_txt)


# In[18]:


temp_num = sent2num(temp_txt)
print(temp_num[:5])


[[6892, 6892, 6892, 16, 63, 611, 369], [6892, 6892, 16, 63, 611, 369, 729], [6892, 16, 63, 611, 369, 729, 253], [16, 63, 611, 369, 729, 253, 449], [63, 611, 369, 729, 253, 449, 50]]


In [14]:

# 根据输入得到标注推断
def predict_num(input_num,input_txt,
                model = model,
                label_dict=label_dict,
                num_dict=num_dict):
    input_num = np.array(input_num)
    predict_prob = model.predict_proba(input_num)
    predict_lable = model.predict_classes(input_num)
    for i , lable in enumerate(predict_lable[:-1]):
        # 如果是首字 ，不可为E, M
        if i==0:
            predict_prob[i, label_dict[u'E']] = 0
            predict_prob[i, label_dict[u'M']] = 0
        # 前字为B，后字不可为B,S
        if lable == label_dict[u'B']:
            predict_prob[i+1,label_dict[u'B']] = 0
            predict_prob[i+1,label_dict[u'S']] = 0
        # 前字为E，后字不可为M,E
        if lable == label_dict[u'E']:
            predict_prob[i+1,label_dict[u'M']] = 0
            predict_prob[i+1,label_dict[u'E']] = 0
        # 前字为M，后字不可为B,S
        if lable == label_dict[u'M']:
            predict_prob[i+1,label_dict[u'B']] = 0
            predict_prob[i+1,label_dict[u'S']] = 0
        # 前字为S，后字不可为M,E
        if lable == label_dict[u'S']:
            predict_prob[i+1,label_dict[u'M']] = 0
            predict_prob[i+1,label_dict[u'E']] = 0
        predict_lable[i+1] = predict_prob[i+1].argmax()
    predict_lable_new = [num_dict[x]  for x in predict_lable]
    result =  [w+'/' +l  for w, l in zip(input_txt,predict_lable_new)]
    return ' '.join(result) + '\n'


# In[26]:


temp = predict_num(temp_num,temp_txt)
print(temp)

国/B 家/M 食/M 药/M 监/M 总/M 局/E 发/B 布/E 通/B 知/E 称/S ，/S 酮/B 康/E 唑/B 口/E 服/B 制/E 剂/B 因/E 存/B 在/E 严/B 重/E 肝/B 毒/E 性/S 不/B 良/E 反/B 应/E ，/S 即/B 日/E 起/S 停/B 止/E 生/B 产/E 销/B 售/E 使/B 用/E 。/S



In [18]:
import codecs
import sys
from os import path
from pickle import dump,load


test_file = './data/icwb2-data/testing/msr_test.utf8'
with open(test_file,'r') as f:
    lines = f.readlines()
    test_texts = [list(line.strip()) for line in lines]


# In[ ]:


test_output = []
for line in test_texts:
    test_num = sent2num(line)
    output_line = predict_num(test_num,input_txt=line)
    test_output.append(output_line)


# In[31]:


with open('./data/icwb2-data/testing/msr_test_output.utf8','w') as f:
    f.writelines(test_output)


# In[32]:


input_file = './data/icwb2-data/testing/msr_test_output.utf8'
output_file = './data/icwb2-data/testing/msr_test.split.tag2word.utf8'


# In[33]:



def character_2_word(input_file, output_file):
    input_data = codecs.open(input_file, 'r', 'utf-8')
    output_data = codecs.open(output_file, 'w', 'utf-8')
    # 4 tags for character tagging: B(Begin), E(End), M(Middle), S(Single)
    for line in input_data.readlines():
        char_tag_list = line.strip().split()
        for char_tag in char_tag_list:
            char_tag_pair = char_tag.split('/')
            char = char_tag_pair[0]
            tag = char_tag_pair[1]
            if tag == 'B':
                output_data.write(' ' + char)
            elif tag == 'M':
                output_data.write(char)
            elif tag == 'E':
                output_data.write(char + ' ')
            else: # tag == 'S'
                output_data.write(' ' + char + ' ')
        output_data.write("\n")
    input_data.close()
    output_data.close()

character_2_word(input_file, output_file)



In [19]:
!./data/icwb2-data/scripts/score data/icwb2-data/gold/msr_training_words.utf8 data/icwb2-data/gold/msr_test_gold.utf8 data/icwb2-data/testing/msr_test.split.tag2word.utf8 > deep.score

In [20]:
!tail -n 50 deep.score

剧烈								剧烈
的								的
碰撞								碰撞
，								，
都								都
将								将
激起								激起
一							      |	一团耀
团							      |	眼
耀眼							      <
的								的
火花								火花
，								，
这								这
就								就
是								是
崭新								崭新
的								的
计算机技术						      |	计算机
							      >	技术
和								和
古老								古老
的								的
中华								中华
文化								文化
的								的
对话								对话
。								。
INSERTIONS:	2
DELETIONS:	2
SUBSTITUTIONS:	7
NCHANGE:	11
NTRUTH:	45
NTEST:	45
TRUE WORDS RECALL:	0.800
TEST WORDS PRECISION:	0.800
=== SUMMARY:
=== TOTAL INSERTIONS:	4303
=== TOTAL DELETIONS:	4382
=== TOTAL SUBSTITUTIONS:	11187
=== TOTAL NCHANGE:	19872
=== TOTAL TRUE WORD COUNT:	106873
=== TOTAL TEST WORD COUNT:	106794
=== TOTAL TRUE WORDS RECALL:	0.854
=== TOTAL TEST WORDS PRECISION:	0.855
=== F MEASURE:	0.855
=== OOV Rate:	0.026
=== OOV Recall Rate:	0.586
=== IV Recall Rate:	0.862
###	data/icwb2-data/testing/msr_test.split.tag2word.utf8	4303	4382	11187	19872	106873	106794	0.854	0.855	0.855	0.026	0.586	0.862
